In [2]:
import socket
import time
import pandas as pd
import os
from bleak import BleakScanner
import asyncio

# Server details
SERVER_HOST = '127.0.0.1'
SERVER_PORT = 5000

# File path for storing Bluetooth device data
EXCEL_FILE_PATH = r'C:\Users\Lenovo\Desktop\bluetooth_devices.xlsx'

# Initialize the Excel file if it doesn't exist
if not os.path.exists(EXCEL_FILE_PATH):
    df = pd.DataFrame(columns=["MAC Address", "Device Name"])
    df.to_excel(EXCEL_FILE_PATH, index=False)

def send_signal_to_csharp(message):
    """Send a message to the C# application."""
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        try:
            s.connect((SERVER_HOST, SERVER_PORT))
            s.sendall(message.encode('utf-8'))
            print(f"Message sent to C#: {message}")
        except Exception as e:
            print("Error sending signal to C# application:", e)

def register_or_login_device(mac_address, device_name):
    """Check if the device is new or existing, update the Excel file, and send a signal to C#."""
    # Load the DataFrame, or create it if it doesn’t exist
    if os.path.exists(EXCEL_FILE_PATH):
        df = pd.read_excel(EXCEL_FILE_PATH)
    else:
        df = pd.DataFrame(columns=["MAC Address", "Device Name"])

    # Check if the MAC address already exists
    if mac_address in df["MAC Address"].values:
        print(f"Device {device_name} with MAC {mac_address} is already registered. Logging in.")
        send_signal_to_csharp("User logged in")  # Send "User logged in" message
    else:
        print(f"Registering new device {device_name} with MAC {mac_address}.")
        new_device = pd.DataFrame({"MAC Address": [mac_address], "Device Name": [device_name]})
        df = pd.concat([df, new_device], ignore_index=True)

        # Try saving the Excel file with retry handling
        for attempt in range(3):
            try:
                df.to_excel(EXCEL_FILE_PATH, index=False)
                print(f"New device {device_name} registered successfully.")
                send_signal_to_csharp("User registered")  # Send "User registered" message
                break
            except PermissionError:
                print(f"Attempt {attempt + 1}: Excel file locked. Retrying...")
                time.sleep(1)

async def scan_bluetooth_devices():
    print("Scanning for Bluetooth devices...")
    devices = await BleakScanner.discover()

    for device in devices:
        print(f"Detected Bluetooth device: Name: {device.name}, MAC: {device.address}")
        register_or_login_device(device.address, device.name)

async def main():
    # Continuously scan for devices
    while True:
        await scan_bluetooth_devices()
        await asyncio.sleep(5)  # Control the scan frequency

# Run the main async function
if __name__ == "__main__":
    try:
        asyncio.run(main())
    except RuntimeError as e:
        print("Asyncio event loop issue:", e)
        await main()


Asyncio event loop issue: asyncio.run() cannot be called from a running event loop
Scanning for Bluetooth devices...
Detected Bluetooth device: Name: None, MAC: 18:4A:D5:CC:AC:AF
Device None with MAC 18:4A:D5:CC:AC:AF is already registered. Logging in.
Message sent to C#: User logged in
Detected Bluetooth device: Name: None, MAC: 7C:C9:0F:56:0D:09
Device None with MAC 7C:C9:0F:56:0D:09 is already registered. Logging in.
Message sent to C#: User logged in
Detected Bluetooth device: Name: None, MAC: 15:AC:1A:B0:9E:50
Device None with MAC 15:AC:1A:B0:9E:50 is already registered. Logging in.
Message sent to C#: User logged in
Detected Bluetooth device: Name: [TV] Samsung 7 Series (55), MAC: FC:03:9F:21:1C:3B
Device [TV] Samsung 7 Series (55) with MAC FC:03:9F:21:1C:3B is already registered. Logging in.
Message sent to C#: User logged in
Scanning for Bluetooth devices...
Detected Bluetooth device: Name: [TV] Samsung 7 Series (55), MAC: FC:03:9F:21:1C:3B
Device [TV] Samsung 7 Series (55) wit

CancelledError: 

In [5]:
import socket
import mediapipe as mp
import cv2
import pickle
import threading
import asyncio
from dollarpy import Point
from bleak import BleakScanner


HOSTNAME = '127.0.0.1'
PORT = 5010


soc = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
soc.bind((HOSTNAME, PORT))
soc.listen(5)
conn , addr = soc.accept()
print("device connected")

def capture_gestures():
    mp_drawing = mp.solutions.drawing_utils
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands()
    with open('gesture_recognizer1.pkl', 'rb') as file:
        recognizer = pickle.load(file)
    points = []
    wrist = []  #0
    thumb_cmc = [] #1
    thumb_mcp = [] #2
    thumb_ip = [] #3
    thumb_tip = [] #4
    index_finger_mcp = [] #5
    index_finger_pip = [] #6
    index_finger_dip = [] #7
    index_finger_tip = [] #8
    middle_finger_mcp = [] #9
    middle_finger_pip = [] #10
    middle_finger_dip = [] #11
    middle_finger_tip = [] #12
    ring_finger_mcp = [] #13
    ring_finger_pip = [] #14
    ring_finger_dip = [] #15
    ring_finger_tip = [] #16
    pinky_mcp = [] #17 
    pinky_pip = [] #18
    pinky_dip = [] #19
    pinky_tip = [] #20


    cap = cv2.VideoCapture(0)
    while cap.isOpened():
        _, frame = cap.read()
        try:
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = hands.process(frame_rgb)
            annotated_image = frame.copy()
            
            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(annotated_image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                    try:
                        wrist.append(Point(hand_landmarks.landmark[0].x, hand_landmarks.landmark[0].y, 1))
                        thumb_cmc.append(Point(hand_landmarks.landmark[1].x, hand_landmarks.landmark[1].y, 1))
                        thumb_mcp.append(Point(hand_landmarks.landmark[2].x, hand_landmarks.landmark[2].y, 1))
                        thumb_ip.append(Point(hand_landmarks.landmark[3].x, hand_landmarks.landmark[3].y, 1))
                        thumb_tip.append(Point(hand_landmarks.landmark[4].x, hand_landmarks.landmark[4].y, 1))
                        index_finger_mcp.append(Point(hand_landmarks.landmark[5].x, hand_landmarks.landmark[5].y, 1))
                        index_finger_pip.append(Point(hand_landmarks.landmark[6].x, hand_landmarks.landmark[6].y, 1))
                        index_finger_dip.append(Point(hand_landmarks.landmark[7].x, hand_landmarks.landmark[7].y, 1))
                        index_finger_tip.append(Point(hand_landmarks.landmark[8].x, hand_landmarks.landmark[8].y, 1))
                        middle_finger_mcp.append(Point(hand_landmarks.landmark[9].x, hand_landmarks.landmark[9].y, 1))
                        middle_finger_pip.append(Point(hand_landmarks.landmark[10].x, hand_landmarks.landmark[10].y, 1))
                        middle_finger_dip.append(Point(hand_landmarks.landmark[11].x, hand_landmarks.landmark[11].y, 1))
                        middle_finger_tip.append(Point(hand_landmarks.landmark[12].x, hand_landmarks.landmark[12].y, 1))
                        ring_finger_mcp.append(Point(hand_landmarks.landmark[13].x, hand_landmarks.landmark[13].y, 1))
                        ring_finger_pip.append(Point(hand_landmarks.landmark[14].x, hand_landmarks.landmark[14].y, 1))
                        ring_finger_dip.append(Point(hand_landmarks.landmark[15].x, hand_landmarks.landmark[15].y, 1))
                        ring_finger_tip.append(Point(hand_landmarks.landmark[16].x, hand_landmarks.landmark[16].y, 1))
                        pinky_mcp.append(Point(hand_landmarks.landmark[17].x, hand_landmarks.landmark[17].y, 1))
                        pinky_pip.append(Point(hand_landmarks.landmark[18].x, hand_landmarks.landmark[18].y, 1))
                        pinky_dip.append(Point(hand_landmarks.landmark[19].x, hand_landmarks.landmark[19].y, 1))
                        pinky_tip.append(Point(hand_landmarks.landmark[20].x, hand_landmarks.landmark[20].y, 1))
                        
                        points = wrist + thumb_cmc + thumb_mcp + thumb_ip + thumb_tip + \
                            index_finger_mcp + index_finger_pip + index_finger_dip + index_finger_tip + \
                                middle_finger_mcp + middle_finger_pip + middle_finger_dip + middle_finger_tip + \
                                    ring_finger_mcp + ring_finger_pip + ring_finger_dip + ring_finger_tip + \
                                        pinky_mcp + pinky_pip + pinky_dip + pinky_tip
                        prediction = recognizer.recognize(points)[0]
                        print(prediction)
                        if prediction:
                            msg_pred = f"{prediction}\n".encode("utf-8")  # Adding a newline at the end of the prediction message
                            conn.send(msg_pred)
                        points = []
                    except Exception as e:
                        print("An error occurred:", str(e))
            annotated_image = cv2.resize(annotated_image, (700, 500))
            cv2.imshow('Output', annotated_image)
        except:
            break
        if cv2.waitKey(1) == ord('q'):
            cap.release()
            cv2.destroyAllWindows()
            cv2.waitKey(100)
            break
    cap.release()
    cv2.destroyAllWindows()

async def scan_for_devices():
    global stop_program
    devices = await BleakScanner.discover()
    for device in devices:
        print(f"Device: {device.name}, MAC Address: {device.address}")
        msg_pred = f"Device: {device.name}, MAC Address: {device.address}\n".encode("utf-8")  # Adding newline at the end
        conn.send(msg_pred)

def scan_in_background():
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    loop.run_until_complete(scan_for_devices())


gesture_thread = threading.Thread(target=capture_gestures)
scanner_thread = threading.Thread(target=scan_in_background)

gesture_thread.start()
scanner_thread.start()

gesture_thread.join()
scanner_thread.join()

conn.send(bytes("q", "utf-8"))
conn.close()
soc.close()

device connected
Device: None, MAC Address: 1F:3C:15:42:68:94
Device: None, MAC Address: B0:E4:5C:32:95:63
Device: None, MAC Address: FC:03:9F:21:1C:3B
Device: None, MAC Address: 69:18:0F:4F:82:C5
Device: None, MAC Address: 68:25:5D:0D:65:A8
Device: None, MAC Address: E3:AF:EA:08:4A:B0
Device: None, MAC Address: 53:EC:5E:08:75:4B
Device: None, MAC Address: 4A:40:57:44:1E:22
Device: None, MAC Address: 69:6F:98:E3:75:66
Device: None, MAC Address: 8C:CE:FD:6A:11:ED


In [13]:
import socket
import mediapipe as mp
import cv2
import pickle
import threading
import asyncio
from dollarpy import Point
from bleak import BleakScanner


HOSTNAME = '127.0.0.1'
PORT = 5010


soc = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
soc.bind((HOSTNAME, PORT))
soc.listen(5)
conn , addr = soc.accept()
print("device connected")

def capture_gestures():
    mp_drawing = mp.solutions.drawing_utils
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands()
    with open('gesture_recognizer1.pkl', 'rb') as file:
        recognizer = pickle.load(file)
    points = []
    wrist = []  #0
    thumb_cmc = [] #1
    thumb_mcp = [] #2
    thumb_ip = [] #3
    thumb_tip = [] #4
    index_finger_mcp = [] #5
    index_finger_pip = [] #6
    index_finger_dip = [] #7
    index_finger_tip = [] #8
    middle_finger_mcp = [] #9
    middle_finger_pip = [] #10
    middle_finger_dip = [] #11
    middle_finger_tip = [] #12
    ring_finger_mcp = [] #13
    ring_finger_pip = [] #14
    ring_finger_dip = [] #15
    ring_finger_tip = [] #16
    pinky_mcp = [] #17 
    pinky_pip = [] #18
    pinky_dip = [] #19
    pinky_tip = [] #20


    cap = cv2.VideoCapture(0)
    while cap.isOpened():
        _, frame = cap.read()
        try:
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = hands.process(frame_rgb)
            annotated_image = frame.copy()
            
            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(annotated_image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                    try:
                        wrist.append(Point(hand_landmarks.landmark[0].x, hand_landmarks.landmark[0].y, 1))
                        thumb_cmc.append(Point(hand_landmarks.landmark[1].x, hand_landmarks.landmark[1].y, 1))
                        thumb_mcp.append(Point(hand_landmarks.landmark[2].x, hand_landmarks.landmark[2].y, 1))
                        thumb_ip.append(Point(hand_landmarks.landmark[3].x, hand_landmarks.landmark[3].y, 1))
                        thumb_tip.append(Point(hand_landmarks.landmark[4].x, hand_landmarks.landmark[4].y, 1))
                        index_finger_mcp.append(Point(hand_landmarks.landmark[5].x, hand_landmarks.landmark[5].y, 1))
                        index_finger_pip.append(Point(hand_landmarks.landmark[6].x, hand_landmarks.landmark[6].y, 1))
                        index_finger_dip.append(Point(hand_landmarks.landmark[7].x, hand_landmarks.landmark[7].y, 1))
                        index_finger_tip.append(Point(hand_landmarks.landmark[8].x, hand_landmarks.landmark[8].y, 1))
                        middle_finger_mcp.append(Point(hand_landmarks.landmark[9].x, hand_landmarks.landmark[9].y, 1))
                        middle_finger_pip.append(Point(hand_landmarks.landmark[10].x, hand_landmarks.landmark[10].y, 1))
                        middle_finger_dip.append(Point(hand_landmarks.landmark[11].x, hand_landmarks.landmark[11].y, 1))
                        middle_finger_tip.append(Point(hand_landmarks.landmark[12].x, hand_landmarks.landmark[12].y, 1))
                        ring_finger_mcp.append(Point(hand_landmarks.landmark[13].x, hand_landmarks.landmark[13].y, 1))
                        ring_finger_pip.append(Point(hand_landmarks.landmark[14].x, hand_landmarks.landmark[14].y, 1))
                        ring_finger_dip.append(Point(hand_landmarks.landmark[15].x, hand_landmarks.landmark[15].y, 1))
                        ring_finger_tip.append(Point(hand_landmarks.landmark[16].x, hand_landmarks.landmark[16].y, 1))
                        pinky_mcp.append(Point(hand_landmarks.landmark[17].x, hand_landmarks.landmark[17].y, 1))
                        pinky_pip.append(Point(hand_landmarks.landmark[18].x, hand_landmarks.landmark[18].y, 1))
                        pinky_dip.append(Point(hand_landmarks.landmark[19].x, hand_landmarks.landmark[19].y, 1))
                        pinky_tip.append(Point(hand_landmarks.landmark[20].x, hand_landmarks.landmark[20].y, 1))
                        
                        points = wrist + thumb_cmc + thumb_mcp + thumb_ip + thumb_tip + \
                            index_finger_mcp + index_finger_pip + index_finger_dip + index_finger_tip + \
                                middle_finger_mcp + middle_finger_pip + middle_finger_dip + middle_finger_tip + \
                                    ring_finger_mcp + ring_finger_pip + ring_finger_dip + ring_finger_tip + \
                                        pinky_mcp + pinky_pip + pinky_dip + pinky_tip
                        prediction = recognizer.recognize(points)[0]
                        print(prediction)
                        if prediction:
                            msg_pred = f"{prediction}\n".encode("utf-8")  # Adding a newline at the end of the prediction message
                            conn.send(msg_pred)
                        points = []
                    except Exception as e:
                        print("An error occurred:", str(e))
            annotated_image = cv2.resize(annotated_image, (700, 500))
            cv2.imshow('Output', annotated_image)
        except:
            break
        if cv2.waitKey(1) == ord('q'):
            cap.release()
            cv2.destroyAllWindows()
            cv2.waitKey(100)
            break
    cap.release()
    cv2.destroyAllWindows()

async def scan_for_devices():
    global stop_program
    devices = await BleakScanner.discover()
    for device in devices:
        print(f"Device: {device.name}, MAC Address: {device.address}")
        msg_pred = f"Device: {device.name}, MAC Address: {device.address}\n".encode("utf-8")  # Adding newline at the end
        conn.send(msg_pred)

def scan_in_background():
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    loop.run_until_complete(scan_for_devices())


gesture_thread = threading.Thread(target=capture_gestures)
scanner_thread = threading.Thread(target=scan_in_background)

gesture_thread.start()
scanner_thread.start()

gesture_thread.join()
scanner_thread.join()

conn.send(bytes("q", "utf-8"))
conn.close()
soc.close()

device connected
Device: None, MAC Address: B0:E4:5C:32:95:63
Device: None, MAC Address: FC:03:9F:21:1C:3B
Device: None, MAC Address: 53:33:3B:EA:CF:45
None
Peace
Peace
Peace
Peace
Peace


   ---------------------------------------- 0.0/525.3 kB ? eta -:--:--
   ---------------------------------------- 0.0/525.3 kB ? eta -:--:--
   ---------------------------------------  524.3/525.3 kB 1.3 MB/s eta 0:00:01
   ---------------------------------------- 525.3/525.3 kB 1.2 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
